In [1]:
import json
import torch
import numpy as np
from tqdm import tqdm
import os
import sys
sys.path.append("../")

import configparser
import psycopg2
import pymysql
import pymysql.cursors as pycursor
import numpy as np
import torch.optim as optim

import time
import glob
from models.concurrency.baselines.gcn_graph_gen import generate_graph
from models.concurrency.baselines.gcn_graph_embedding import load_all_data
from models.concurrency.baselines.gcn_train import train_one_batch, eval_one_batch
from models.concurrency.baselines.gcn_constant import args
from models.concurrency.baselines.gcn_v2 import get_model, get_optimizer
np.set_printoptions(suppress=True)

In [2]:
data_dir = "/Users/ziniuw/Desktop/research/perf_prediction/workload-performance/pmodel_data/imdb/"
dataset = "sample-plan"
n_run_id = 4
print("Loading all data graphs")
all_features, all_edge_idx, all_edge_weight, all_labels, all_root_idx_bit_map = load_all_data(data_dir, dataset, n_run_id)
n_graphs = len(all_features)
print(f"In total, {n_graphs} number of graphs")

Loading all data graphs
In total, 835 number of graphs


In [3]:
model = get_model(feature_num=all_features[0].shape[-1], hidden=args.hidden, nclass=args.node_dim, nlayers=args.n_layers, dropout=args.dropout)
optimizer = get_optimizer(model=model, lr=1e-4, weight_decay=2e-5)

In [8]:
i = 1
pred = model(all_features[i], all_edge_idx[i], all_edge_weight[i]*0)
np.sum(np.isnan(pred.detach().numpy()))

0

In [9]:
pred

tensor([[ 85.3933],
        [ 94.6440],
        [ 95.2601],
        ...,
        [119.8099],
        [145.2255],
        [ 93.2462]], grad_fn=<AddmmBackward0>)

In [11]:
all_edge_idx[i][:, 60: 80]

tensor([[65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,
         84, 85],
        [64, 65, 65, 66, 66, 69, 71, 72, 73, 74, 74, 75, 75, 77, 77, 79, 79, 80,
         81, 81]])

In [13]:
all_edge_weight[i][60: 80]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])

In [ ]:
a = all_labels[0].detach().numpy()[:20, 0]

In [10]:
torch.stack((pred[-20:, 0], all_labels[0][-20:, 0]), axis=1)

tensor([[7.8130e+01, 1.0000e-03],
        [1.1375e+02, 1.0845e-01],
        [7.8987e+01, 1.0000e-03],
        [8.6377e+01, 1.0327e-01],
        [8.6238e+01, 3.3790e-03],
        [1.2371e+02, 9.8294e-02],
        [8.5717e+01, 1.0000e-03],
        [1.1203e+02, 7.6671e-02],
        [9.8461e+01, 1.5310e-03],
        [1.2710e+02, 1.2430e+00],
        [1.4431e+02, 1.2581e+00],
        [1.7708e+02, 1.2524e+00],
        [1.8185e+02, 8.7539e-01],
        [4.3742e+01, 2.3621e+01],
        [8.0644e+01, 1.7485e+01],
        [9.9855e+01, 1.7476e+01],
        [1.5150e+02, 1.2366e+01],
        [1.1981e+02, 2.4867e+00],
        [1.4523e+02, 4.0987e-02],
        [9.3246e+01, 2.2874e-02]], grad_fn=<StackBackward0>)

In [ ]:
train_one_batch(
            model, all_labels[0], all_features[0], all_edge_idx[0], all_edge_weight[0], optimizer
        )

In [14]:
num_epoch = 100
eval_every = 5
save_best=True
save_path=None
all_idx = np.random.permutation(n_graphs)
train_idx = all_idx[0: int(0.85 * n_graphs)]
val_idx = all_idx[int(0.85 * n_graphs):]
best_val_loss = np.infty
for epoch in range(num_epoch):
    model.train()
    curr_epoch_train_idx = np.random.permutation(train_idx)
    batch_loss = 0
    #bl = train_one_batch(
     #       model, all_labels[0], all_features[0], all_edge_idx[0], all_edge_weight[0], optimizer
      #  )
    #batch_loss += bl
    if epoch < 100:
        loss_func="mse"
    else:
        loss_func="qloss"
    for i in tqdm(curr_epoch_train_idx):
        # per-batch training
        bl = train_one_batch(
            model, all_labels[i], all_features[i], all_edge_idx[i], all_edge_weight[i], optimizer, loss_func=loss_func
        )
        batch_loss += bl
    if epoch % eval_every == 0:
        print(
            f"Epoch {epoch} ======================================================"
        )
        print(f"Training loss: {batch_loss/len(curr_epoch_train_idx)}")
        model.eval()
        val_pred = []
        val_label = []
        val_loss = 0
        for i in val_idx:
            vl, pred, label = eval_one_batch(
                model,
                all_labels[i],
                all_features[i],
                all_edge_idx[i],
                all_edge_weight[i],
                all_root_idx_bit_map[i],
            )
            val_loss += vl
            val_pred.append(pred)
            val_label.append(label)
        val_pred = np.maximum(np.concatenate(val_pred), 1e-3)
        val_label = np.maximum(np.concatenate(val_label), 1e-3)
        print(f"Validation loss: {val_loss / len(val_idx)}")
        abs_error = np.abs(val_pred - val_label)
        q_error = np.maximum(val_pred / val_label, val_label / val_pred)
        for p in [50, 90, 95]:
            p_a = np.percentile(abs_error, p)
            p_q = np.percentile(q_error, p)
            print(f"{p}% absolute error is {p_a}, q-error is {p_q}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if save_best and save_path is not None:
                torch.save(model.state_dict(), save_path)

100%|████████████████████████████████████████████████████████████████████████| 709/709 [00:01<00:00, 503.09it/s]


Epoch 0 ======================================================
Training loss: 8.34627811505025
Validation loss: 7.0657799119482645
50% absolute error is 11.112966060638428, q-error is 5.801250457763672
90% absolute error is 127.35953826904299, q-error is 41.863964843750004
95% absolute error is 210.56489791870106, q-error is 75.62653236389154


100%|████████████████████████████████████████████████████████████████████████| 709/709 [00:01<00:00, 572.23it/s]


Epoch 5 ======================================================
Training loss: 8.250366938867852
Validation loss: 6.971002506838907
50% absolute error is 14.0413236618042, q-error is 5.636868238449097
90% absolute error is 123.51944580078127, q-error is 44.91866798400883
95% absolute error is 206.9116142272949, q-error is 85.63552513122558


100%|████████████████████████████████████████████████████████████████████████| 709/709 [00:01<00:00, 557.40it/s]


...

100%|████████████████████████████████████████████████████████████████████████| 709/709 [00:01<00:00, 561.76it/s]


Epoch 90 ======================================================
Training loss: 7.732178253171543
Validation loss: 6.985638186147343
50% absolute error is 11.721551895141602, q-error is 5.229491710662842
90% absolute error is 125.08281478881835, q-error is 38.980287933349636
95% absolute error is 209.12036056518542, q-error is 70.68481826782222


100%|████████████████████████████████████████████████████████████████████████| 709/709 [00:01<00:00, 564.06it/s]


Epoch 95 ======================================================
Training loss: 7.716723963356566
Validation loss: 6.931726301000707
50% absolute error is 13.884275913238525, q-error is 5.247399806976318
90% absolute error is 122.96008377075196, q-error is 41.32335433959961
95% absolute error is 206.35063247680645, q-error is 80.9823638916014


100%|████████████████████████████████████████████████████████████████████████| 709/709 [00:01<00:00, 561.42it/s]


In [ ]:
q_error = np.maximum(val_pred / val_label, val_pred / val_label)

In [13]:
np.stack((val_pred[:100], val_label[:100]), axis=1)

array([[  1.9766109 ,   0.12929654],
       [  1.9760406 ,  29.356312  ],
       [  1.9778183 , 177.6734    ],
       [  1.9790504 ,  40.7558    ],
       [  1.9805267 ,  17.379774  ],
       [  1.9811513 ,   0.34537005],
       [  1.981139  ,   0.70237017],
       [  1.9808834 ,   0.47925687],
       [  1.9812686 ,   0.31960416],
       [  1.9812438 ,  39.52201   ],
       [  1.9812696 ,   2.682489  ],
       [  1.9812849 ,   0.3374312 ],
       [  1.981386  ,  14.057903  ],
       [  1.9818618 ,   0.16178107],
       [  1.9820964 ,  19.11175   ],
       [  1.9827602 ,   2.5012956 ],
       [  1.982805  ,   0.06894326],
       [  1.9828861 ,   5.701987  ],
       [  1.9830921 ,   4.943334  ],
       [  1.9830577 ,  16.020132  ],
       [  1.9832227 ,   3.7853863 ],
       [  1.9835241 ,   1.4430311 ],
       [  1.9835289 ,   0.73183274],
       [  1.9836681 ,  11.282608  ],
       [  1.9838827 ,   0.74304795],
       [  1.9847829 ,   9.524972  ],
       [  1.9840925 ,   9.003878  ],
 

In [ ]:
val_label

In [ ]:
data_path = '/Users/ziniuw/Desktop/research/perf_prediction/workload-performance/pmodel_data/job/'

edge_dim = 25000 # upper bound of edges
node_dim = 300 # upper bound of nodes

mp_optype = {'Aggregate': 0, 'Nested Loop': 1, 'Index Scan': 2, 'Hash Join': 3, 'Seq Scan': 4, 'Hash': 5, 'Update': 6} # operator types in the queries

oid = 0 # operator number

In [ ]:
class DictParser(configparser.ConfigParser):
    def read_dict(self):
        d = dict(self._sections)
        for k in d:
            d[k] = dict(d[k])
        return d

cf = DictParser()
cf.read("/Users/ziniuw/Desktop/research/perf_prediction/workload-performance/config.ini", encoding="utf-8")
config_dict = cf.read_dict()


def parse_knob_config(config_dict):
    _knob_config = config_dict["knob_config"]
    for key in _knob_config:
        _knob_config[key] = json.loads(str(_knob_config[key]).replace("\'", "\""))
    return _knob_config
knob_config = parse_knob_config(config_dict)

In [ ]:
knob_config

In [ ]:
"""
workloads = glob.glob(data_path + "sample-plan-*")

start_time = time.time()
num_graphs = 3000
for wid in range(num_graphs):
    st = time.time()

    vmatrix, ematrix, mergematrix = generate_graph(wid, connect=False)
    #print("[graph {}]".format(wid), "time:{}; #-vertex:{}, #-edge:{}".format(time.time() - st, len(vmatrix), len(ematrix)))
    
    with open(data_path + "graph/" + "sample-plan-" + str(wid) + ".content", "w") as wf:
        for v in vmatrix:
            wf.write(str(v[0]) + "\t" + str(v[1]) + "\t" + str(v[2]) + "\t" + str(v[3]) + "\t" + str(v[4]) + "\n")
    with open(data_path + "graph/" + "sample-plan-" + str(wid) + ".cites", "w") as wf:
        for e in ematrix:
            wf.write(str(e[0]) + "\t" + str(e[1]) + "\t" + str(e[2]) + "\n")
    #with open(data_path + "graph/" + "sample-plan-" + str(wid) + ".merges", "w") as wf:
     #   for e in mergematrix:
      #      wf.write(str(e[0]) + "\t" + str(e[1]) + "\t" + str(e[2]) + "\n")

end_time = time.time()

print("Total Time:{}".format(end_time - start_time))
"""

In [ ]:
workloads = glob.glob(save_dir + "sample-plan-0-*-nodes.npy")

In [ ]:
workloads

In [ ]:
save_dir = "/Users/ziniuw/Desktop/research/perf_prediction/workload-performance/pmodel_data/imdb/"
for wid in range(2):
    # Load data 
    adj, features, labels, root_idx_bit_map, idx_train, idx_val, idx_test = load_data(path = save_dir, dataset = f"sample-plan-0-{wid}" )
    print(adj.shape, features.shape, labels.shape, root_idx_bit_map.shape)

In [ ]:
labels[idx_val]

In [ ]:
output = model(features, adj)

In [ ]:
preds = output[idx_val].max(1)[1].type_as(labels)

In [ ]:
output.max(1)[0]

In [ ]:
preds

In [ ]:
model = GCN(nfeat=24,
            nhid=10,
            nclass=128, 
            dropout=0.1)    

optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
iteration_num = 3
num_epoch = 30
for wid in range(iteration_num):
    print("[graph {}]".format(wid))
    # Load data 
    adj, features, labels, root_idx_bit_map, idx_train, idx_val, idx_test = load_data(path = save_dir, dataset = f"sample-plan-0-{wid}")
    labels = labels.reshape(-1, 1)
    
    # Model Training
    ok_times = 0
    t_total = time.time()
    for epoch in range(num_epoch):
        # print(features.shape, adj.shape)
        if epoch == num_epoch - 1:
            debug = True
        else:
            debug = False
        loss_train = train(epoch, labels, features, adj, idx_train, idx_val, model, optimizer, debug)
        if loss_train < 0.002:
            ok_times += 1
        if ok_times >= 20:
            break    
        
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))